# Shark Attacks dataset. Cleaning using pandas and regex
Dataset to clean: https://www.kaggle.com/teajay/global-shark-attacks/version/1

In [1]:
import pandas as pd
import re
data = pd.read_csv("input/GSAF5.csv", encoding = "ISO-8859-15")

### Table Overview

In [2]:
data.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


In [3]:
data.dtypes

Case Number               object
Date                      object
Year                       int64
Type                      object
Country                   object
Area                      object
Location                  object
Activity                  object
Name                      object
Sex                       object
Age                       object
Injury                    object
Fatal (Y/N)               object
Time                      object
Species                   object
Investigator or Source    object
pdf                       object
href formula              object
href                      object
Case Number.1             object
Case Number.2             object
original order             int64
Unnamed: 22               object
Unnamed: 23               object
dtype: object

In [4]:
null_cols = data.isnull().sum()
null_cols
null_cols[null_cols > 0]

Country                     43
Area                       402
Location                   496
Activity                   527
Name                       200
Sex                        567
Age                       2681
Injury                      27
Fatal (Y/N)                 19
Time                      3213
Species                   2934
Investigator or Source      15
href formula                 1
href                         3
Unnamed: 22               5991
Unnamed: 23               5990
dtype: int64

### 1st clean: Drop invalid columns
- Columns 'Unnamed: 22' & 'Unnamed: 23' are not referenced in the description of the dataset and doesn't contain any relevant information. Proceed to drop them.
- Columns 'Case Number.1' & 'Case Number.2' are duplicates of 'Case Number'. Proceed to drop them.
- "Date" cannot be normalized. We drop it to later get the dates from the Case Number column on a structured way.

In [5]:
data = data.drop(['Unnamed: 22','Unnamed: 23', 'Case Number.1', 'Case Number.2', 'Date'], axis=1)

### 2nd clean: Renaming columns
- Some names of the columns aren't clean or clear enough. Below the list of columns renamed
    - Sex: remove a blank space at the end.
    - Country: Changed to Area, since several entrances refer to seas or regions broader than a country.

In [6]:
data.rename(columns={
    'Sex ':'Sex', 
    'Country': 'Place'
    }, inplace=True)

### 3rd clean: 

Among the total 5900 events registered, only 137 happened before 1700. To evaluate only statistically relevant data, events registered before 1700 will not be considered

In [7]:
data = data[data['Year'] > 1700]
print(data.shape)

(5852, 19)


### 4th clean: Unifying categories
    - Sex: Typo found on 2 entrances. Fixed.
    - Country: We have reduced the list of countries from the original set of 197 categories, to 174. For that purpose we have used both regular expressions and manual replacement.

In [8]:
data.replace({'Sex': {'M ': 'M'}}, inplace=True)

In [9]:
type(data['Sex'])

pandas.core.series.Series

In [10]:
data['Sex'].value_counts()

M      4723
F       572
lli       1
.         1
N         1
Name: Sex, dtype: int64

In [11]:
#remove end ?
#remove start/end blank spaces
#remove 2nd country after /
data.replace(regex={
    r'\?':'', 
    r'\s\/\s[A-Z\s]+': '', 
    r'\s$':'', r'^\s':''
}, inplace=True)

In [12]:
# On column Place, manually fixed some duplicates
data.replace({'Place': { 'UNITED ARAB EMIRATES (UAE)':'UNITED ARAB EMIRATES', 
'Fiji':'FIJI', 'ST. MAARTIN':'ST. MARTIN', 
'Seychelles':'SEYCHELLES', 
'Sierra Leone':'SIERRA LEONE', 
'St Helena': 'ST HELENA', 
'ENGLAND': 'UNITED KINGDOM', 
'SCOTLAND': 'UNITED KINGDOM'}
}, inplace=True)

In [13]:
len(set(data['Place']))

174

### 5th clean: Normalizing column Activity
Reduce from the original 1418 unique values on Activity to 5: 'Surfing', 'Swimming', 'Fishing', 'Diving' & 'Others'.

In [14]:
data.rename(columns={'Activity':'unActivity'}, inplace=True)
data_activity = data['unActivity']
activity = []
for e in data_activity:
    if re.search(r'Surf[\w\s\,]+|surf[\w\s\,]+|[\w\s\,]+surf[\w\s\,]+', str(e)):
        e = 'Surfing'
    elif re.search(r'Fish[\w\s\,]+|fish[\w\s\,]+|[\w\s\,]+fish[\w\s\,]+', str(e)):
        e = 'Fishing'
    elif re.search(r'Spear[\w\s\,]+|spear[\w\s\,]+|[\w\s\,]+spear[\w\s\,]+', str(e)):
        e = 'Fishing'
    elif re.search(r'Swim[\w\s\,]+|swim[\w\s\,]+|[\w\s\,]+swim[\w\s\,]+', str(e)):
        e = 'Swimming'
    elif re.search(r'Div[\w\s\,]+|div[\w\s\,]+|[\w\s\,]+div[\w\s\,]+', str(e)):
        e = 'Diving'
    else: e = 'Others'
    activity.append(e)
data['Activity'] = activity
data = data.drop(['unActivity'], axis=1)

In [15]:
data['Activity'].value_counts()

Others      2098
Surfing     1157
Fishing     1108
Swimming    1014
Diving       475
Name: Activity, dtype: int64

### 6th clean: Create a new column for dates, getting the information from the column 'Case Number'

In [16]:
data['Date'] = data['Case Number']

In [17]:
data['Date'].replace(regex = {r'.[A-Za-z]$':''}, inplace = True)

### 7th clean: Create a new column for the month, extracting it from the 'Case Number' column.

In [18]:
data['Month'] = [e[5:7] for e in data['Case Number']]

### 8th Clean: Normalizing the hour, keeping only the values that correspond to a 24h value

In [19]:
data['Time'] = data['Time'].replace(regex = {r'\s[\w\-\d\/\()]+|\-[\w\-\d\/]+|j$|^\>|^\<':'', r'h':':'})
hour = []
time = data['Time']
for e in time:
    if re.search(r'\d{2}\:\d{2}', str(e)) == None:
        e = 'Unknown'
    print(e)
    hour.append(e)
data['Hour'] = hour

13:00
11:00
10:43
Unknown
Unknown
Unknown
15:15
14:30
15:40
Unknown
Unknown
Unknown
Unknown
15:00
14:00
17:00
16:00
Unknown
Unknown
Unknown
11:30
Unknown
Unknown
Unknown
12:00
19:05
Unknown
Unknown
Unknown
Unknown
Unknown
11:00
Unknown
Unknown
Unknown
Unknown
10:00
Unknown
14:30
22:00
16:20
14:34
11:00
Unknown
15:25
14:55
06:00
Unknown
17:30
15:00
11:30
08:30
11:30
Unknown
Unknown
Unknown
16:00
16:00
15:45
12:00
18:00
17:46
Unknown
13:20
15:49
Unknown
07:00
17:30
08:00
12:00
Unknown
Unknown
10:45
Unknown
Unknown
Unknown
11:00
19:00
Unknown
11:00
Unknown
Unknown
13:30
Unknown
Unknown
Unknown
Unknown
19:00
16:00
15:00
Unknown
12:30
13:20
Unknown
Unknown
14:00
11:30
09:30
Unknown
Unknown
Unknown
Unknown
11:30
Unknown
Unknown
Unknown
07:00
Unknown
Unknown
10:30
Unknown
Unknown
Unknown
Unknown
Unknown
18:15
11:00
11:00
14:00
04:00
Unknown
14:50
15:00
14:30
13:50
Unknown
19:20
11:30
11:00
16:20
10:25
Unknown
16:50
13:00
10:00
08:30
16:20
Unknown
10:45
16:45
15:52
12:30
07:30
19:00
15:00
06:1

12:00
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
14:30
Unknown
16:45
Unknown
Unknown
Unknown
Unknown
Unknown
18:00
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
08:00
14:00
15:00
Unknown
09:00
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
10:00
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
11:30
13:30
Unknown
Unknown
Unknown
17:00
Unknown
Unknown
11:00
Unknown
Unknown
20:00
16:20
Unknown
Unknown
Unknown
14:45
Unknown
Unknown
Unknown
Unknown
Unknown
14:00
Unknown
10:30
Unknown
13:15
Unknown
Unknown
15:35
Unknown
Unknown
11:30
11:00
Unknown
Unknown
Unknown
10:20
16:30
Unknown
Unknown
Unknown
15:00
Unknown
23:00
Unknown
Unknown
Unknown
Unknown
10:00
Unknown
11:30
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
16:00
14:00
14:00
Unknown
16:00
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
12:00
Unknown
Unknown
Unknown
Unknown
Un

Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
05:00
Unknown
Unknown
Unknown
Unknown
Unknown
10:30
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
05:00
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
12:00
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
16:00
Unknown
Unknown
16:30
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unkn

###  9th clean: Change column types
- Change the column Fatal (Y/N) to a boolean, normalizing all the entries to True or False.
- The few unknown values have been trated as non fatal.

In [20]:
data.rename(columns={ 'Fatal (Y/N)' : 'Fatal'}, inplace=True)
data = data.replace({'Fatal':
             { 'N' : '0', 
               'Y' : '1',
               'n' : '0',
               'y' : '1',
              'UNKNOWN' : '0',
              'F' : '0', 
              '#VALUE!' : '0'
             }})
data['Fatal'].astype(bool)


0       True
1       True
2       True
3       True
4       True
        ... 
5847    True
5848    True
5849    True
5850    True
5851    True
Name: Fatal, Length: 5852, dtype: bool

### Final Table
- 10 columns with categorized data selected to further analysis.
- Excluded from this final table the columns with unique values.

In [21]:
final_table = data[['Date', 'Year', 'Month', 'Hour', 'Place', 'Area', 'Activity', 'Sex', 'Fatal']]
final_table.to_csv("output/cleaned_data_GSAF5.csv")
display(final_table)

,Date,Year,Month,Hour,Place,Area,Activity,Sex,Fatal
0,2016.09.18,2016,09,13:00,USA,Florida,Surfing,M,0
1,2016.09.18,2016,09,11:00,USA,Florida,Surfing,M,0
2,2016.09.18,2016,09,10:43,USA,Florida,Surfing,M,0
3,2016.09.17,2016,09,Unknown,AUSTRALIA,Victoria,Surfing,M,0
4,2016.09.15,2016,09,Unknown,AUSTRALIA,Victoria,Surfing,M,0
...,...,...,...,...,...,...,...,...,...
5847,1742.12.17,1742,12,Unknown,NaN,NaN,Swimming,M,1
5848,1738.04.06,1738,04,Unknown,ITALY,Sicily,Swimming,M,1
5849,1733.00.00,1733,00,Unknown,ICELAND,Bardestrand,Others,NaN,NaN
5850,1721.06.00,1721,06,Unknown,ITALY,Sardinia,Swimming,M,1


Until here the cleaning process. Go to data_views.ipynb for the analysis of the data:
https://github.com/castares/pandas-data-cleaning/blob/master/data_views.ipynb
